Purpose: Scrape past tournaments and get new matchups for current tournament

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')


In [2]:
year='2017'

regionNames1=['east','west','midwest','south','national']
regionNames2=['east','southeast','southwest','west','national']

Get Seeds and initial matchups

In [4]:
for year in [str(yr) for yr in np.arange(2017,2016,-1)]:

    url='https://www.sports-reference.com/cbb/postseason/'+year+'-ncaa.html'
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html5lib")

    region={}
    
    regionNames=[]
    divs=soup.findAll("div", {"data-controls": '#brackets'})
    for reg in divs[0].find_all('div'):
        print(reg.find('a').text)
        regionNames.append(reg.find('a').text.lower().replace(' ','').replace('.',''))
    regionNames[-1]='national'
    
    for regionName in regionNames:
        divs=soup.findAll("div", {"id": regionName})[0]
        divround=divs.find_all('div',{'class': 'round'})[0]

        teamNames=[]
        for t in divround.find_all('div'):
            tname=t.find('a').text.lower().replace(' ','-')
            if tname not in teamNames:
                teamNames.append(tname)
        teamNames
#         region[regionName]=dict(zip(seedList,teamNames))

    # regionNames=['east']
    region={}
    initRound={}
    for regionName in regionNames:
        initRound[regionName]={}
        divs=soup.findAll("div", {"id": regionName})[0]
        rounds=divs.find_all('div',{'class': 'round'})
        region[regionName]={}
        for iround in range(len(rounds)-1):
            region[regionName][iround]=[]
            divround=rounds[iround]

            teamNames=[]
            seedList=[]
            for t in divround.find_all('div'):
                tname=t.find('a').text.lower().replace(' ','-')
                tname=t.find('a')['href'].split('/')[3].lower().replace(' ','-')#.replace('-st-','-st.-')
                if tname not in teamNames:
                    seedList.append(t.find('span').text)
                    teamNames.append(tname)

            winnerNames=[]
            for t in divround.find_all('div', {'class': 'winner'}):
                tname=t.find('a').text.lower().replace(' ','-')
                tname=t.find('a')['href'].split('/')[3].lower().replace(' ','-')#.replace('-st-','-st.-')
                if tname not in winnerNames:
                    winnerNames.append(tname)

                    
            if regionName=='national':
                roundNumber = iround + 1 + 4
            else:
                roundNumber = iround + 1
            gameList=[]
            # fill list for games
            for t1,t2,s1,s2 in zip(teamNames[0::2],teamNames[1::2],seedList[0::2],seedList[1::2]):
                if t1 in winnerNames:
                    result = 1
                if t2 in winnerNames: 
                    result = 0

                gameList.append([t1,t2,result,roundNumber,regionName])
                # fill list for only initial teams
                if iround==0:
                    initRound[regionName][s1]=t1
                    initRound[regionName][s2]=t2
#                     initRound.append([s1,t1])
#                     initRound.append([s2,t2])
                
            region[regionName][iround] = gameList

    gameList=[]
    for regionName in region.keys():
        for iround in region[regionName].keys():
            for gameOutcome in region[regionName][iround]:
                gameList.append(gameOutcome)

    all_games=gameList

    team_data=pd.read_csv('data/combined_team_stats_'+year+'.csv')

    name=[]
    for r in team_data.itertuples():
        name.append(r.sched_url.split('/')[3])
    team_data['name']=name

    combinedDF=[]
    print('Looping over games in ',year)
    for game in all_games:
        team_one = team_data[team_data['name'] == game[0]]
        column1 = team_one.columns[1:]    
        team_one[column1].values[0]

        team_two = team_data[team_data['name'] == game[1]]
        column2 = team_two.columns[1:]    
        team_two[column2].values[0]

        combinedDF.append(list(team_one[column1].values[0])+list(team_two[column2].values[0])+[game[2],game[3],game[4]])
    combinedColumns=[c+'_1' for c in column1]+[c+'_2' for c in column2]+['outcome','round','region']

    game_data=pd.DataFrame(combinedDF,columns=combinedColumns)
    game_data.to_csv("data/games/tourn_games_" + year + ".csv")
    # game_data
    teamList=[]
    for regionName in initRound.keys():
        region=initRound[regionName]
        if regionName=='national':
            continue
        for iseed in region.keys():
            teamList.append([regionName,iseed,region[iseed]])
    tourneyTeams = pd.DataFrame(teamList,columns=['regionName','seed','teamName'])
    tourneyTeams.to_csv("data/team/tourn_teams_" + year + ".csv")


East
Midwest
South
West
Final Four
Looping over games in  2017


In [8]:
game_data[['name_1','name_2','round','outcome']]

,name_1,name_2,round,outcome
0,villanova,mount-st-marys,1,1
1,wisconsin,virginia-tech,1,1
2,virginia,north-carolina-wilmington,1,1
3,florida,east-tennessee-state,1,1
4,southern-methodist,southern-california,1,0
5,baylor,new-mexico-state,1,1
6,south-carolina,marquette,1,1
7,duke,troy,1,1
8,villanova,wisconsin,2,0
9,virginia,florida,2,0


In [21]:
game

['duke', 'campbell', 1, 1, 'east']

In [10]:
year='2003'
team_data=pd.read_csv('data/combined_team_stats_'+year+'.csv')
[t if 'troy' in t else None for t in team_data.name]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [5]:
divround

<div class="round"><div> <!-- game -->
	<div class="winner"> <!-- team -->
		
		<span>1</span>
               <a href="/cbb/schools/north-carolina/2017.html">UNC</a>
		<a href="/cbb/boxscores/2017-04-03-north-carolina.html">71</a>
		
	</div>
	
	<div> <!-- team -->
		
		<span>1</span>
               <a href="/cbb/schools/gonzaga/2017.html">Gonzaga</a>
		<a href="/cbb/boxscores/2017-04-03-north-carolina.html">65</a>
		
	</div>
	<span><a href="/cbb/boxscores/2017-04-03-north-carolina.html">at Glendale, AZ</a></span>
	
</div>
</div>